In [1]:
%matplotlib inline
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pink_utils as pu
import pandas as pd
import astropy.units as u

from dask import delayed
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from astropy.coordinates import SkyCoord, search_around_sky
from pink_utils import Annotation


In [2]:
FIRST_PIX = 1.8*u.arcsecond # Pixel size of FIRST survey. Square pixels
FIRST_FWHM = 5*u.arcsecond / FIRST_PIX
FIRST_SIG = FIRST_FWHM / 2.355

In [3]:
df        = pd.read_csv('../FIRST_F1W1_95_5_Sources.csv')
transform = pu.transform('../Small/FIRST_F1W1_95_5_Small_Transform.bin')
ed        = pu.heatmap('../Small/FIRST_F1W1_95_5_Small_Similarity.bin')
som       = pu.som('../Small/FIRST_F1W1_95_5_L1_SOM_Small_5.bin')
# images    = pu.image_binary('../FIRST_F1W1_95_5_imgs.bin')

with open('../Small/FIRST_F1W1_95_5_L1_SOM_Small_5_Features-table.pkl','rb') as infile:
    annotations = pickle.load(infile)

In [4]:
print(som.file_head)
# print(images.file_head)
print(transform.file_head)
print(ed.file_head)
print(df.shape)
NEURON_SIZE = som.file_head[-1] # Equal dimensions 

(2, 12, 12, 1, 118, 118)
(178859, 12, 12, 1)
(178859, 12, 12, 1)
(178859, 29)


In [6]:
def bmu_pos_to_index(pos, width=20):
    return pos[0]*width+pos[1]


@delayed
def list_to_df(results):
    return pd.DataFrame(results)

@delayed
def process_df(count, index, row, ed, transform):
    pred_results = []

    if count != index:
        print('Mistmatch', index, count)

    cen_pos = SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)
    
    bmu_pos    = ed.get_bmu(index)
    bmu_ed     = ed.get_bmu_ed(index)
    bmu_index  = bmu_pos_to_index(bmu_pos, width=som.file_head[1])
    try:
        trans_info = transform.get_neuron_transform(index, bmu_pos)
    except:
        print(index)
        
        
    # Annotate_map_features.py script recorded positions around the 
    # incorrect convention
    key = (bmu_pos[1], bmu_pos[0]) + (0,)
    bmu = annotations[key]
    
    # Transform annotated clicks 
    r_feature_trans = bmu.transform_clicks(trans_info, channel=0)
    
    fake_trans = [np.random.randint(2), np.random.uniform(0,2*np.pi)]
    fake_r_feature_trans = bmu.transform_clicks(fake_trans, channel=0)
    
    for click_count, (r_click, fake_r_click) in enumerate(zip(r_feature_trans, fake_r_feature_trans)):
        
        dx, dy = r_click
        fdx, fdy = fake_r_click
        
        comp_pos = pu.estimate_position(cen_pos, dx, dy, pix_scale=FIRST_PIX)
        fake_pos = pu.estimate_position(cen_pos, fdx, fdy, pix_scale=FIRST_PIX)
        
        pred_results.append({'pred_ra':  comp_pos.ra.deg,
                        'pred_dec': comp_pos.dec.deg,
                        'pred_sep': comp_pos.separation(cen_pos).to(u.deg).value,
                        'fake_ra':  fake_pos.ra.deg,
                        'fake_dec': fake_pos.dec.deg,
                        'fake_sep': fake_pos.separation(cen_pos).to(u.deg).value,                        
                        'first_ra':  row['RA'],
                        'first_dec': row['DEC'],
                        'first_index': index,
                        'bmu_ed': bmu_ed,
                        'bmu_index': bmu_index,
                        'bmu_pos':bmu_pos,
                        'bmu_clicks': len(r_feature_trans),
                        'click_count': click_count})
        
        
        

plt.close('all')        

ms = ['o','*','>']
c  = ['red', 'blue', 'black']
cmap = 'Greys'

results = []

results = [process_df(count, index, row, ed, transform) for (count, (index, row)) in tqdm(enumerate(df.iterrows()))]
with ProgressBar():
    results = list_to_df(results).compute( num_workers=6, scheduler='processes')



         

178859it [00:20, 8555.11it/s]


[                                        ] | 0% Completed |  6min  6.7s

Process ForkPoolWorker-15:
Process ForkPoolWorker-18:
Process ForkPoolWorker-17:
Process ForkPoolWorker-19:
Process ForkPoolWorker-16:
Process ForkPoolWorker-20:


[                                        ] | 0% Completed |  6min  6.8s


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)

KeyboardInterrupt: 

  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/connection.py", line 411, in _recv_bytes
    return self._recv(size)
  File "/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [9]:
radio_df = pd.DataFrame(results)

In [10]:
radio_df.to_csv('Predicting_radio.csv')

In [11]:
radio_df.shape

(207054, 13)

In [8]:
transform.file_head[1:]

(12, 12, 1)